In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the training data
training_data = pd.read_excel("top 5+ stock 2019-2023.xlsx")

# Normalize parameter names in training data
training_data["Parameters"] = training_data["Parameters"].str.lower().str.strip()

# Transform the data for training
training_data_melted = training_data.melt(
    id_vars=["Company_name", "Parameters"], 
    var_name="Year", 
    value_name="Value"
)

# Pivot the data to have years as columns
training_data_pivot = training_data_melted.pivot_table(
    index=["Company_name", "Parameters"],
    columns="Year",
    values="Value"
).reset_index()

# Extract features and target columns
features = training_data_pivot.iloc[:, 2:-1].values  # All years except the last one
target = training_data_pivot.iloc[:, -1].values  # The last year as the target

# Normalize the features and target
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)
target_scaled = scaler.fit_transform(target.reshape(-1, 1))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    features_scaled, target_scaled, test_size=0.2, random_state=42
)

# Define the neural network model
model = Sequential([
    Dense(128, activation="relu", input_dim=X_train.shape[1]),
    Dropout(0.2),
    Dense(64, activation="relu"),
    Dropout(0.2),
    Dense(1, activation="linear")
])

model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32
)

# Load the forecast data
forecast_data = pd.read_excel("top 5 2024_2033.xlsx")

# Normalize parameter names in forecast data
forecast_data["Parameters"] = forecast_data["Parameters"].str.lower().str.strip()

# Define the parameter to forecast
parameter_to_forecast = "repurchase of common stock".lower().strip()

# Check if the parameter exists in the forecast data
if parameter_to_forecast not in forecast_data["Parameters"].values:
    available_parameters = forecast_data["Parameters"].unique()
    raise ValueError(
        f"No matching data found in forecast data for '{parameter_to_forecast}'.\n"
        f"Available parameters: {available_parameters}"
    )

# Filter the forecast data for the desired parameter
forecast_data_filtered = forecast_data[forecast_data["Parameters"] == parameter_to_forecast]

# Prepare the forecast features
forecast_features = forecast_data_filtered.iloc[:, 2:].values
forecast_features_scaled = scaler.transform(forecast_features)

# Predict future values
predicted_values_scaled = model.predict(forecast_features_scaled)

# Inverse scale the predictions
predicted_values = scaler.inverse_transform(predicted_values_scaled)

# Add the predictions to the forecast data
forecast_data_filtered.loc[:, "Predicted Values"] = predicted_values

# Save the updated forecast data
forecast_data_filtered.to_csv("forecasted_data_with_predictions.csv", index=False)

print("Forecasting complete. Results saved to 'forecasted_data_with_predictions.csv'.")
